In [ ]:
!pip install transformers[torch] torch

In [ ]:
dataset = []

with open('drive/MyDrive/Colab Notebooks/data/geo-reviews-dataset-2023.tskv') as data:
    for line in data:
        sample = {}

        for substr in line.split("\t"):
            for idx, char in enumerate(substr):
                if char == '=':
                    key = substr[:idx].strip()
                    value = substr[idx + 1:].strip()

                    sample[key] = value.replace(r'\n', ' ')

        dataset.append(sample)

In [ ]:
unique = set()

for d in dataset:
    for r in d['rubrics'].split(';'):
        if r not in unique:
            unique.add(r.lower())

from pprint import pprint

pprint(unique)

{'3d-услуги',
 'gps-оборудование',
 'ip-телефония',
 'it-компания',
 'pr-агентство',
 'wi-fi хот-спот',
 'аварийная служба',
 'аварийно-спасательное оборудование и техника',
 'аварийный комиссар',
 'авиакомпания',
 'авиационное и аэродромное оборудование',
 'автоаксессуары',
 'автоакустика',
 'автоателье',
 'автоаукцион',
 'автобусные билеты',
 'автобусные перевозки',
 'автобусный парк',
 'автовокзал, автостанция',
 'автодиагностика',
 'автодром',
 'автоклуб, мотоклуб',
 'автокондиционеры',
 'автокосметика, автохимия',
 'автокресла',
 'автоломбард',
 'автоматизация документооборота',
 'автоматизация производств',
 'автоматизация ресторанов',
 'автоматические двери и ворота',
 'автомобильная парковка',
 'автомобильные грузоперевозки',
 'автомобильные ключи и брелоки',
 'автомобильные отопители',
 'автомобильные прицепы',
 'автомобильные радиаторы',
 'автомобильные тахографы',
 'автомобильный завод',
 'автомоечное оборудование',
 'автомойка',
 'автоподбор',
 'авторазбор',
 'авторынок',
 

In [ ]:
rubrics_wl = {
    'кафе', 'антикафе', 'кофейня', 'кондитерская', 'кофейный автомат',
    'ресторан', 'быстрое питание', 'доставка еды и обедов',
    'пекарня', 'безалкогольный бар', 'безалкогольные напитки',
    'винная ярмарка', 'комбинаты питания', 'кофе с собой',
    'столовая', 'бар, паб', 'суши-бар', 'торты', 'магазин пива',
    'пиццерия', 'спортивное питание', 'фудкорт', 'банкетный зал', 'магазин сыров',
    'торты на заказ'
}

len(rubrics_wl)

25

In [ ]:
filtered = []

for d in dataset:
    if 'name_ru' not in d.keys():
        continue
    for r in d['rubrics'].split(';'):
        if r.lower() in rubrics_wl:
            filtered.append(d)
            break

In [ ]:
len(filtered)

124660

In [ ]:
print(*dataset[:5], sep='\n')

{'address': 'Екатеринбург, ул. Московская / ул. Волгоградская / ул. Печатников', 'name_ru': 'Московский квартал', 'rating': '3.', 'rubrics': 'Жилой комплекс', 'text': 'Московский квартал 2. Шумно : летом по ночам дикие гонки. Грязно : кругом стройки, невозможно открыть окна (16 этаж! ), вечно по району летает мусор. Детские площадки убогие, на большой площади однотипные конструкции. Очень дорогая коммуналка. Часто срабатывает пожарная сигнализация. Жильцы уже не реагируют. В это время, обычно около часа, не работают лифты. Из плюсов - отличная планировка квартир ( Московская 194 ), на мой взгляд. Ремонт от застройщика на 3-. Окна вообще жуть - вместо вентиляции. По соотношению цена/качество - 3.'}
{'address': 'Московская область, Электросталь, проспект Ленина, 29', 'name_ru': 'Продукты Ермолино', 'rating': '5.', 'rubrics': 'Магазин продуктов;Продукты глубокой заморозки;Магазин мяса, колбас', 'text': 'Замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное 

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
# Note: by default pip will install pydantic v1 (because of other pre-isntalled
# packages). Deserialization may fail if v2 is installed on target machine
!pip install -U qdrant-client

In [ ]:
from qdrant_client.http.models import PointStruct

docs = list(map(lambda x: x['text'], filtered))
metadata = list(
    map(
        lambda x: {
            'rubrics': x['rubrics'].split(';'),
            'rating': x['rating'],
            'address': x['address'],
            'name': x['name_ru']}, filtered))
ids = list(range(len(filtered)))

batch_size = 1024
idx = 0

points = []

while idx < len(docs):
    d = docs[idx:idx + batch_size]
    embeddings = model.encode(d)

    for i, e in enumerate(embeddings):
        points.append(
            PointStruct(
                id=i + idx,
                vector=list(map(float, e)),
                payload={
                    'address': metadata[i + idx]['address'],
                    'rubrics': metadata[i + idx]['rubrics'],
                    'name': metadata[i + idx]['name'],
                    'rating': int(metadata[i + idx]['rating'][:-1] if metadata[i + idx]['rating'].endswith('.') else metadata[i + idx]['rating']),
                    'text': d[i]
                }))

    idx += batch_size

In [ ]:
import pickle

with open('drive/MyDrive/Colab Notebooks/data/points.pickle', 'wb') as f:
    pickle.dump(points, f)

In [ ]:
del points

In [ ]:
import gc

gc.collect()

328